In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from ha_utils import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 2 s
Wall time: 2.66 s


In [881]:

from datetime import datetime
from urllib.parse import urlparse, parse_qs
%run ../load_magic/dataframes.py
from html_analysis import ElementAnalysis
from IPython.display import clear_output

ea = ElementAnalysis()

In [6]:

def generate_child_strs(verbose=False):
    file_path = '../data/html/indeed_email.html'
    page_soup = get_page_soup(file_path)
    css_selector = 'body > table > tbody > tr > td > a > table > tbody > tr > td > a'
    link_soups_list = page_soup.select(css_selector)
    for link_soup in link_soups_list:
        url_str = link_soup['href']
        page_soup = get_page_soup(url_str)
        page_title = page_soup.find('title').string.strip()
        file_name = re.sub(r'\W+', ' ', page_title).strip().replace(' ', '_')
        jk = parse_qs(urlparse(url_str).query).get('jk', [''])[0]
        if len(jk):
            file_name = f'{jk}_{file_name}.html'
        else:
            # file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}.html'
            file_name = f'{file_name}.html'
        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if not os.path.isfile(file_path):
            with open(file_path, 'w', encoding=s.encoding_type) as f:
                if verbose:
                    print(f'Saving to {file_path}')
                f.write('<html><head><title>')
                f.write(page_title)
                f.write('</title></head><body>')
                row_div_list = page_soup.find_all(name='div', attrs={'class': ['jobsearch-JobComponent-description']})
                for div_soup in row_div_list:
                    f.write(str(div_soup))
                f.write('</body></html>')
        clear_output(wait=True)
        file_name_id = cu.get_filename_id(file_name, verbose=verbose)
        row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
        for div_soup in row_div_list:
            child_strs_list = ea.ha.get_navigable_children(div_soup, [])
            child_tags_list = ea.ha.construct_child_tags_list(child_strs_list)
            for sequence_order, (navigable_parent, header_tag) in enumerate(zip(child_strs_list, child_tags_list)):
                clear_output(wait=True)
                header_tag_id = cu.get_headertag_id(header_tag, verbose=verbose)
                header_tag_sequence_id = cu.get_headertagsequence_id(file_name_id, header_tag_id, sequence_order, verbose=verbose)
                cu.ensure_headertagsequence_filename_relationship(file_name_id, verbose=verbose)
                cu.ensure_headertagsequence_headertag_relationship(header_tag_id, verbose=verbose)
                
                navigable_parent_id = cu.get_navigableparent_id(navigable_parent, verbose=verbose)
                cu.ensure_headertag_navigableparent_relationship(header_tag_id, navigable_parent_id, verbose=verbose)
                
                navigable_parent_sequence_id = cu.get_navigableparentsequence_id(file_name_id, navigable_parent_id, sequence_order, verbose=verbose)
                cu.ensure_navigableparentsequence_filename_relationship(file_name_id, verbose=verbose)
                cu.ensure_navigableparentsequence_navigableparent_relationship(navigable_parent_id, verbose=verbose)
                
                yield navigable_parent

In [7]:

CHILD_STRS_LIST = list(generate_child_strs(verbose=False))

In [62]:

ZMQInteractiveShell_obj = get_ipython()
NAVIGABLE_PARENT_IS_HEADER_DICT = s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
def get_dictionary_code():
    output_str = ''
    tag_str = CHILD_STRS_LIST.pop()
    while tag_str in NAVIGABLE_PARENT_IS_HEADER_DICT:
        tag_str = CHILD_STRS_LIST.pop()
    output_str += f'\n# {len(CHILD_STRS_LIST)} to go\n'
    if "'" in tag_str:
        tag_str = tag_str.replace('"', '\\"')
        output_str += f'tag_str = "{tag_str}"\n'
    else:
        output_str += f"tag_str = '{tag_str}'\n"
    output_str += 'NAVIGABLE_PARENT_IS_HEADER_DICT[tag_str] = False\n'
    output_str += 'print(len(NAVIGABLE_PARENT_IS_HEADER_DICT.keys()))\n'
    output_str += 's.store_objects(NAVIGABLE_PARENT_IS_HEADER_DICT=NAVIGABLE_PARENT_IS_HEADER_DICT)\n'
    output_str += 'navigable_parent = cu.clean_text(tag_str)\n'
    output_str += 'if NAVIGABLE_PARENT_IS_HEADER_DICT[tag_str]:\n'
    output_str += '    cypher_str = cu.set_is_header1_cypher_str.format(navigable_parent)\n'
    output_str += 'else:\n'
    output_str += '    cypher_str = cu.set_is_header0_cypher_str.format(navigable_parent)\n'
    output_str += 'print(cypher_str)\n'
    output_str += 'with cu.driver.session() as session:\n'
    output_str += '    session.write_transaction(cu.do_cypher_tx, cypher_str)'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)